## Bribes research

In [3]:
import math

import pandas as pd
import requests
from IPython.core.display_functions import display

from notebooks.bribes_research.constants import BRIBES_URL
from notebooks.bribes_research.constants import DASHBOARD_URL
from notebooks.bribes_research.constants import PROTOCOL_AURA
from notebooks.bribes_research.constants import PROTOCOL_CVX
from notebooks.bribes_research.constants import TABLE_COLUMNS
from notebooks.bribes_research.helpers import calculate_metrics
from notebooks.bribes_research.helpers import fetch_token_prices

pd.set_option('display.max_rows', 500)
BADGER_BRIBE = 42000
STEP = 2000
BADGER_LOW_BOUND = 2000
BADGER_UPPER_BOUND = BADGER_BRIBE
# HH and Votium take 4% on all bribes
FEE = 0.04  # in %

REBALANCING_STEP = 0.01  # in %

table_rows = []
token_prices = fetch_token_prices(['badger-dao'])
badger_price = token_prices['badger-dao']['usd']

cvx_bribes = requests.post(BRIBES_URL, json={'platform': "votium", 'protocol': "cvx-crv"}).json()
cvx_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-votium"}).json()
aura_bribes = requests.post(BRIBES_URL, json={'platform': "hh", 'protocol': "aura-bal"}).json()
aura_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-aura"}).json()
total_bribes_cvx = sum([bribe['amountDollars'] for bribe in cvx_bribes['epoch']['bribes']])
dollar_spent_on_bribes_cvx = cvx_dash['dashboard']['rewardPerDollarBribe']

total_bribes_aura = sum([bribe['amountDollars'] for bribe in aura_bribes['epoch']['bribes']])
dollar_spent_on_bribes_aura = aura_dash['dashboard']['rewardPerDollarBribe']
for bribe in range(BADGER_LOW_BOUND, BADGER_UPPER_BOUND, STEP):
    badger_bribe_in_dollar = bribe * badger_price
    fees = badger_bribe_in_dollar * FEE
    badger_bribe_in_dollar_after_fees = badger_bribe_in_dollar - fees
    cvx_dilution_50, cvx_in_out_50 = calculate_metrics(badger_bribe_in_dollar_after_fees / 2, total_bribes_cvx,
                                                       dollar_spent_on_bribes_cvx)
    aura_dilution_50, aura_in_out_50 = calculate_metrics(badger_bribe_in_dollar_after_fees / 2, total_bribes_aura,
                                                         dollar_spent_on_bribes_aura)

    # Rebalance here
    cvx_in_out_floating = cvx_in_out_50
    aura_in_out_floating = aura_in_out_50

    cvx_bribe_portion = aura_bribe_portion = badger_bribe_in_dollar_after_fees / 2
    cvx_dilution = None
    aura_dilution = None
    while not math.isclose(cvx_in_out_floating, aura_in_out_floating, abs_tol=0.5):
        if cvx_in_out_floating > aura_in_out_floating:
            # Take bribe portion from aura and give it to cvx
            bribe_stolen = aura_bribe_portion * REBALANCING_STEP
            aura_bribe_portion -= bribe_stolen
            cvx_bribe_portion += bribe_stolen
        elif aura_in_out_floating > cvx_in_out_floating:
            # Take bribe portion from cvx and give it to aura
            bribe_stolen = aura_bribe_portion * REBALANCING_STEP
            cvx_bribe_portion -= bribe_stolen
            aura_bribe_portion += bribe_stolen
        if cvx_bribe_portion <= 0 or aura_bribe_portion <= 0:
            break
        cvx_dilution, cvx_in_out_floating = calculate_metrics(cvx_bribe_portion, total_bribes_cvx,
                                                              dollar_spent_on_bribes_cvx)
        aura_dilution, aura_in_out_floating = calculate_metrics(aura_bribe_portion, total_bribes_aura,
                                                                dollar_spent_on_bribes_aura)

    table_rows.append(
        [PROTOCOL_CVX, badger_bribe_in_dollar_after_fees, fees, round(cvx_bribe_portion / badger_bribe_in_dollar_after_fees * 100, 2),
         cvx_bribe_portion, cvx_dilution, cvx_in_out_floating]
    )
    table_rows.append(
        [PROTOCOL_AURA, badger_bribe_in_dollar_after_fees, fees, round(aura_bribe_portion / badger_bribe_in_dollar_after_fees * 100, 2),
         aura_bribe_portion, aura_dilution, aura_in_out_floating]
    )
dataframe = pd.DataFrame(data=table_rows, columns=TABLE_COLUMNS)
display(dataframe)

,Protocol,BADGER total bribe in $,Fees,Bribes Portion %,Bribes Portion,Dilution,$I/O
0,CVX,8198.4,341.6,-0.34,-27.724400,0.001994,149.538059
1,AURA,8198.4,341.6,100.34,8226.124400,2.596557,247.636318
2,CVX,16396.8,683.2,-0.34,-55.448799,0.003987,149.535078
3,AURA,16396.8,683.2,100.34,16452.248799,5.193113,241.034891
4,CVX,24595.2,1024.8,-0.34,-83.173199,0.005981,149.532097
5,AURA,24595.2,1024.8,100.34,24678.373199,7.789670,234.433465
6,CVX,32793.6,1366.4,-0.34,-110.897599,0.007975,149.529115
7,AURA,32793.6,1366.4,100.34,32904.497599,10.386226,227.832038
8,CVX,40992.0,1708.0,-0.34,-138.621999,0.009968,149.526134
9,AURA,40992.0,1708.0,100.34,41130.621999,12.982783,221.230611
